SetUp

In [1]:
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.4 LTS"
PRETTY_NAME="Ubuntu 22.04.4 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.4 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [2]:
!find /usr/local -name 'libcublasLt.so.*'

/usr/local/lib/python3.12/dist-packages/nvidia/cublas/lib/libcublasLt.so.12
/usr/local/cuda-12.5/targets/x86_64-linux/lib/libcublasLt.so.12.5.3.2
/usr/local/cuda-12.5/targets/x86_64-linux/lib/libcublasLt.so.12


In [3]:
# Check CUDA version (important for downloading correct cuTENSOR build)
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [4]:
# Example: download cuTENSOR 2.0 for Linux x86_64 CUDA 11.x
# (Update the link depending on CUDA version you saw above)
!wget https://developer.download.nvidia.com/compute/cutensor/2.0.0/local_installers/cutensor-local-repo-ubuntu2004-2.0.0_1.0-1_amd64.deb

# Extract
!sudo dpkg -i cutensor-local-repo-ubuntu2004-2.0.0_1.0-1_amd64.deb
!sudo cp /var/cutensor-local-repo-ubuntu2004-2.0.0/cutensor-*-keyring.gpg /usr/share/keyrings/
!sudo apt-get update
!sudo apt-get -y install libcutensor2 libcutensor-dev libcutensor-doc

--2025-09-15 09:48:47--  https://developer.download.nvidia.com/compute/cutensor/2.0.0/local_installers/cutensor-local-repo-ubuntu2004-2.0.0_1.0-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.59.88.2, 23.59.88.16
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.59.88.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 534317950 (510M) [application/x-deb]
Saving to: ‘cutensor-local-repo-ubuntu2004-2.0.0_1.0-1_amd64.deb’

cutensor-local-repo 100%[===================>] 509.56M   167MB/s    in 3.1s    

2025-09-15 09:48:50 (167 MB/s) - ‘cutensor-local-repo-ubuntu2004-2.0.0_1.0-1_amd64.deb’ saved [534317950/534317950]

Selecting previously unselected package cutensor-local-repo-ubuntu2004-2.0.0.
(Reading database ... 126374 files and directories currently installed.)
Preparing to unpack cutensor-local-repo-ubuntu2004-2.0.0_1.0-1_amd64.deb ...
Unpacking cutensor-local-repo-ubuntu2004-2.0.0 (1.0-1)

In [5]:
!dpkg -l | grep cutensor

ii  cutensor-local-repo-ubuntu2004-2.0.0   1.0-1                                   amd64        cutensor-local repository configuration files
ii  libcutensor-dev                        2.2.0.0-1                               amd64        cuTensor native dev links, headers
ii  libcutensor-doc                        2.2.0.0-1                               amd64        cuTensor documentation
ii  libcutensor2                           2.2.0.0-1                               amd64        cuTensor native runtime libraries


In [6]:
!ls /usr/include/cutensor*

/usr/include/cutensor.h  /usr/include/cutensorMg.h

/usr/include/cutensor:
types.h


In [7]:
%%writefile 00_test_cutensor.cu
#include <cutensor.h>
#include <cuda_runtime.h>
#include <iostream>

int main() {
    cutensorHandle_t handle;
    cutensorStatus_t status = cutensorCreate(&handle);

    if (status == CUTENSOR_STATUS_SUCCESS) {
        std::cout << "✅ cuTENSOR initialized successfully!" << std::endl;
    } else {
        std::cout << "❌ cuTENSOR init failed: " << status << std::endl;
    }

    cutensorDestroy(handle); // cleanup
    return 0;
}

Writing 00_test_cutensor.cu


In [8]:
!nvcc test_cutensor.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcublasLt -lcublas -o test_cutensor

!./test_cutensor

cc1plus: fatal error: test_cutensor.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./test_cutensor: No such file or directory


In [9]:
!find /usr -name "libcublasLt.so*"

/usr/local/lib/python3.12/dist-packages/nvidia/cublas/lib/libcublasLt.so.12
/usr/local/cuda-12.5/targets/x86_64-linux/lib/libcublasLt.so
/usr/local/cuda-12.5/targets/x86_64-linux/lib/stubs/libcublasLt.so
/usr/local/cuda-12.5/targets/x86_64-linux/lib/libcublasLt.so.12.5.3.2
/usr/local/cuda-12.5/targets/x86_64-linux/lib/libcublasLt.so.12


BASIC LINEAR ALGEBRA

Scalars, Vectors, Matrices, and Tensors

In [10]:
%%writefile 01_cutensor_basics.cu
#include <cutensor.h>
#include <cuda_runtime.h>
#include <iostream>
#include <vector>

int main() {
    // Initialize cuTENSOR
    cutensorHandle_t handle;
    cutensorStatus_t status = cutensorCreate(&handle);

    if (status == CUTENSOR_STATUS_SUCCESS) {
        std::cout << "✅ cuTENSOR initialized successfully!" << std::endl;
    } else {
        std::cout << "❌ cuTENSOR init failed: " << status << std::endl;
        return -1;
    }

    // --- 1. Scalar (just a single number) ---
    float scalar = 3.14f;
    std::cout << "Scalar example: " << scalar << std::endl;

    // --- 2. Vector (1D array) ---
    int n = 4;
    std::vector<float> h_vector = {1.0, 2.0, 3.0, 4.0};
    float* d_vector;
    cudaMalloc(&d_vector, n * sizeof(float));
    cudaMemcpy(d_vector, h_vector.data(), n * sizeof(float), cudaMemcpyHostToDevice);
    std::cout << "Vector example: [1, 2, 3, 4]" << std::endl;

    // --- 3. Matrix (2D array) ---
    int rows = 2, cols = 3;
    std::vector<float> h_matrix = {
        1, 2, 3,
        4, 5, 6
    };
    float* d_matrix;
    cudaMalloc(&d_matrix, rows * cols * sizeof(float));
    cudaMemcpy(d_matrix, h_matrix.data(), rows * cols * sizeof(float), cudaMemcpyHostToDevice);
    std::cout << "Matrix example (2x3): [[1,2,3],[4,5,6]]" << std::endl;

    // --- 4. Tensor (3D array, e.g., RGB image 2x2x3) ---
    int dimX = 2, dimY = 2, dimC = 3;
    std::vector<float> h_tensor = {
        // pixel (0,0)
        255, 0, 0,   // Red
        // pixel (0,1)
        0, 255, 0,   // Green
        // pixel (1,0)
        0, 0, 255,   // Blue
        // pixel (1,1)
        255, 255, 0  // Yellow
    };
    float* d_tensor;
    cudaMalloc(&d_tensor, dimX * dimY * dimC * sizeof(float));
    cudaMemcpy(d_tensor, h_tensor.data(), dimX * dimY * dimC * sizeof(float), cudaMemcpyHostToDevice);
    std::cout << "Tensor example (2x2x3 RGB image)" << std::endl;

    // Cleanup
    cudaFree(d_vector);
    cudaFree(d_matrix);
    cudaFree(d_tensor);
    cutensorDestroy(handle);

    return 0;
}

Writing 01_cutensor_basics.cu


In [11]:
!nvcc cutensor_basics.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcublasLt -lcublas -o cutensor_basics

!./cutensor_basics

cc1plus: fatal error: cutensor_basics.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./cutensor_basics: No such file or directory


Vector Operations

In [12]:
%%writefile 02_vector_ops.cu
#include <iostream>
#include <vector>
#include <cmath>

// --- Helper: print vector ---
void printVector(const std::vector<float>& v, const std::string& name) {
    std::cout << name << " = [ ";
    for (auto x : v) std::cout << x << " ";
    std::cout << "]" << std::endl;
}

// --- Vector Addition ---
std::vector<float> add(const std::vector<float>& a, const std::vector<float>& b) {
    std::vector<float> result(a.size());
    for (size_t i = 0; i < a.size(); i++) result[i] = a[i] + b[i];
    return result;
}

// --- Vector Subtraction ---
std::vector<float> subtract(const std::vector<float>& a, const std::vector<float>& b) {
    std::vector<float> result(a.size());
    for (size_t i = 0; i < a.size(); i++) result[i] = a[i] - b[i];
    return result;
}

// --- Scalar Multiplication ---
std::vector<float> scalarMultiply(const std::vector<float>& a, float s) {
    std::vector<float> result(a.size());
    for (size_t i = 0; i < a.size(); i++) result[i] = a[i] * s;
    return result;
}

// --- Dot Product ---
float dot(const std::vector<float>& a, const std::vector<float>& b) {
    float result = 0.0f;
    for (size_t i = 0; i < a.size(); i++) result += a[i] * b[i];
    return result;
}

// --- Cross Product (3D only) ---
std::vector<float> cross(const std::vector<float>& a, const std::vector<float>& b) {
    return {
        a[1]*b[2] - a[2]*b[1],
        a[2]*b[0] - a[0]*b[2],
        a[0]*b[1] - a[1]*b[0]
    };
}

// --- L1 Norm (Manhattan) ---
float l1Norm(const std::vector<float>& a) {
    float sum = 0.0f;
    for (auto x : a) sum += std::fabs(x);
    return sum;
}

// --- L2 Norm (Euclidean) ---
float l2Norm(const std::vector<float>& a) {
    float sum = 0.0f;
    for (auto x : a) sum += x * x;
    return std::sqrt(sum);
}

int main() {
    // Example vectors
    std::vector<float> v1 = {1, 2, 3};
    std::vector<float> v2 = {4, 5, 6};

    printVector(v1, "v1");
    printVector(v2, "v2");

    // Addition & Subtraction
    printVector(add(v1, v2), "v1 + v2");
    printVector(subtract(v1, v2), "v1 - v2");

    // Scalar multiplication
    printVector(scalarMultiply(v1, 2.0f), "2 * v1");

    // Dot product
    std::cout << "Dot(v1, v2) = " << dot(v1, v2) << std::endl;

    // Cross product (3D only)
    printVector(cross(v1, v2), "v1 x v2");

    // Norms
    std::cout << "L1 norm of v1 = " << l1Norm(v1) << std::endl;
    std::cout << "L2 norm of v1 = " << l2Norm(v1) << std::endl;

    return 0;
}

Writing 02_vector_ops.cu


In [13]:
!nvcc vector_ops.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcublasLt -lcublas -o vector_ops

!./vector_ops

cc1plus: fatal error: vector_ops.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./vector_ops: No such file or directory


cuBlas

In [14]:
%%writefile 03_vector_ops_lib.cu
#include <iostream>
#include <vector>
#include <cmath>
#include <cublas_v2.h>
#include <cuda_runtime.h>

// Helper: check CUDA errors
#define CUDA_CHECK(x) if((x) != cudaSuccess){ \
    std::cerr << "CUDA error at " << __LINE__ << std::endl; return -1; }

#define CUBLAS_CHECK(x) if((x) != CUBLAS_STATUS_SUCCESS){ \
    std::cerr << "cuBLAS error at " << __LINE__ << std::endl; return -1; }

int main() {
    // Example vectors (3D for cross product)
    int n = 3;
    std::vector<float> h_v1 = {1.0f, 2.0f, 3.0f};
    std::vector<float> h_v2 = {4.0f, 5.0f, 6.0f};

    float *d_v1, *d_v2;
    CUDA_CHECK(cudaMalloc(&d_v1, n * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_v2, n * sizeof(float)));

    CUDA_CHECK(cudaMemcpy(d_v1, h_v1.data(), n * sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_v2, h_v2.data(), n * sizeof(float), cudaMemcpyHostToDevice));

    // cuBLAS handle
    cublasHandle_t handle;
    CUBLAS_CHECK(cublasCreate(&handle));

    // --- 1. Scalar Multiplication (cublasSscal) ---
    float alpha = 2.0f;
    CUBLAS_CHECK(cublasSscal(handle, n, &alpha, d_v1, 1));
    CUDA_CHECK(cudaMemcpy(h_v1.data(), d_v1, n * sizeof(float), cudaMemcpyDeviceToHost));
    std::cout << "2 * v1 = [ ";
    for (auto x : h_v1) std::cout << x << " ";
    std::cout << "]" << std::endl;

    // Reset v1 for other ops
    h_v1 = {1.0f, 2.0f, 3.0f};
    CUDA_CHECK(cudaMemcpy(d_v1, h_v1.data(), n * sizeof(float), cudaMemcpyHostToDevice));

    // --- 2. Vector Addition/Subtraction (cublasSaxpy) ---
    // v3 = v1 + v2
    std::vector<float> h_result(n);
    float *d_result;
    CUDA_CHECK(cudaMalloc(&d_result, n * sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_result, d_v2, n * sizeof(float), cudaMemcpyDeviceToDevice)); // copy v2 → result
    alpha = 1.0f;
    CUBLAS_CHECK(cublasSaxpy(handle, n, &alpha, d_v1, 1, d_result, 1));
    CUDA_CHECK(cudaMemcpy(h_result.data(), d_result, n * sizeof(float), cudaMemcpyDeviceToHost));
    std::cout << "v1 + v2 = [ ";
    for (auto x : h_result) std::cout << x << " ";
    std::cout << "]" << std::endl;

    // v3 = v1 - v2
    CUDA_CHECK(cudaMemcpy(d_result, d_v1, n * sizeof(float), cudaMemcpyDeviceToDevice)); // copy v1 → result
    alpha = -1.0f;
    CUBLAS_CHECK(cublasSaxpy(handle, n, &alpha, d_v2, 1, d_result, 1));
    CUDA_CHECK(cudaMemcpy(h_result.data(), d_result, n * sizeof(float), cudaMemcpyDeviceToHost));
    std::cout << "v1 - v2 = [ ";
    for (auto x : h_result) std::cout << x << " ";
    std::cout << "]" << std::endl;

    // --- 3. Dot Product (cublasSdot) ---
    float dot_result;
    CUBLAS_CHECK(cublasSdot(handle, n, d_v1, 1, d_v2, 1, &dot_result));
    std::cout << "Dot(v1, v2) = " << dot_result << std::endl;

    // --- 4. Cross Product (manual, not in cuBLAS) ---
    // cuBLAS does not have cross product; implement with simple kernel
    float h_cross[3] = {
        h_v1[1]*h_v2[2] - h_v1[2]*h_v2[1],
        h_v1[2]*h_v2[0] - h_v1[0]*h_v2[2],
        h_v1[0]*h_v2[1] - h_v1[1]*h_v2[0]
    };
    std::cout << "v1 x v2 = [ " << h_cross[0] << " " << h_cross[1] << " " << h_cross[2] << " ]" << std::endl;

    // --- 5. Norms ---
    float norm1, norm2;
    // L1 norm (cublasSasum)
    CUBLAS_CHECK(cublasSasum(handle, n, d_v1, 1, &norm1));
    std::cout << "L1 norm of v1 = " << norm1 << std::endl;

    // L2 norm (cublasSnrm2)
    CUBLAS_CHECK(cublasSnrm2(handle, n, d_v1, 1, &norm2));
    std::cout << "L2 norm of v1 = " << norm2 << std::endl;

    // Cleanup
    cublasDestroy(handle);
    cudaFree(d_v1);
    cudaFree(d_v2);
    cudaFree(d_result);

    return 0;
}

Writing 03_vector_ops_lib.cu


In [15]:
!nvcc vector_ops_lib.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcublasLt -lcublas -o vector_ops_lib

!./vector_ops_lib

cc1plus: fatal error: vector_ops_lib.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./vector_ops_lib: No such file or directory


Matrix Operations

In [16]:
%%writefile 04_matrix_ops_lib.cu
#include <iostream>
#include <vector>
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <cusolverDn.h>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<std::endl; return -1;}
#define CUBLAS_CHECK(x) if((x)!=CUBLAS_STATUS_SUCCESS){ \
    std::cerr<<"cuBLAS error at "<<__LINE__<<std::endl; return -1;}
#define CUSOLVER_CHECK(x) if((x)!=CUSOLVER_STATUS_SUCCESS){ \
    std::cerr<<"cuSOLVER error at "<<__LINE__<<std::endl; return -1;}

int main() {
    int m=2, n=2;
    std::vector<float> h_A = {1,2,3,4}; // row-major 2x2
    std::vector<float> h_B = {5,6,7,8};
    std::vector<float> h_C(m*n);

    float *d_A,*d_B,*d_C;
    CUDA_CHECK(cudaMalloc(&d_A, m*n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_B, m*n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_C, m*n*sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_A,h_A.data(),m*n*sizeof(float),cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_B,h_B.data(),m*n*sizeof(float),cudaMemcpyHostToDevice));

    cublasHandle_t cublasH;
    cusolverDnHandle_t cusolverH;
    CUBLAS_CHECK(cublasCreate(&cublasH));
    CUSOLVER_CHECK(cusolverDnCreate(&cusolverH));

    float alpha=1.0f, beta=0.0f;

    // --- 1. Matrix Addition: C = A + B
    CUBLAS_CHECK(cublasSgeam(cublasH, CUBLAS_OP_N, CUBLAS_OP_N,
                             m, n, &alpha, d_A, m, &alpha, d_B, m,
                             d_C, m));
    CUDA_CHECK(cudaMemcpy(h_C.data(), d_C, m*n*sizeof(float), cudaMemcpyDeviceToHost));
    std::cout<<"A + B = [ "; for(auto x:h_C) std::cout<<x<<" "; std::cout<<"]"<<std::endl;

    // --- 2. Matrix Subtraction: C = A - B
    float neg=-1.0f;
    CUBLAS_CHECK(cublasSgeam(cublasH, CUBLAS_OP_N, CUBLAS_OP_N,
                             m, n, &alpha, d_A, m, &neg, d_B, m,
                             d_C, m));
    CUDA_CHECK(cudaMemcpy(h_C.data(), d_C, m*n*sizeof(float), cudaMemcpyDeviceToHost));
    std::cout<<"A - B = [ "; for(auto x:h_C) std::cout<<x<<" "; std::cout<<"]"<<std::endl;

    // --- 3. Scalar Multiplication: A = 2*A
    alpha=2.0f;
    CUBLAS_CHECK(cublasSscal(cublasH, m*n, &alpha, d_A, 1));
    CUDA_CHECK(cudaMemcpy(h_C.data(), d_A, m*n*sizeof(float), cudaMemcpyDeviceToHost));
    std::cout<<"2 * A = [ "; for(auto x:h_C) std::cout<<x<<" "; std::cout<<"]"<<std::endl;

    // Reset A
    h_A = {1,2,3,4};
    CUDA_CHECK(cudaMemcpy(d_A,h_A.data(),m*n*sizeof(float),cudaMemcpyHostToDevice));

    // --- 4. Matrix Multiplication: C = A * B
    alpha=1.0f; beta=0.0f;
    CUBLAS_CHECK(cublasSgemm(cublasH, CUBLAS_OP_N, CUBLAS_OP_N,
                             m, n, m,
                             &alpha, d_A, m, d_B, m, &beta, d_C, m));
    CUDA_CHECK(cudaMemcpy(h_C.data(), d_C, m*n*sizeof(float), cudaMemcpyDeviceToHost));
    std::cout<<"A * B = [ "; for(auto x:h_C) std::cout<<x<<" "; std::cout<<"]"<<std::endl;

    // --- 5. Transpose: C = A^T
    alpha=1.0f; beta=0.0f;
    CUBLAS_CHECK(cublasSgeam(cublasH, CUBLAS_OP_T, CUBLAS_OP_N,
                             m, n, &alpha, d_A, m, &beta, d_A, m, d_C, m));
    CUDA_CHECK(cudaMemcpy(h_C.data(), d_C, m*n*sizeof(float), cudaMemcpyDeviceToHost));
    std::cout<<"A^T = [ "; for(auto x:h_C) std::cout<<x<<" "; std::cout<<"]"<<std::endl;

    // --- 6. Identity Matrix (2x2) ---
    std::vector<float> h_I = {1,0,0,1};
    std::cout<<"I = [ "; for(auto x:h_I) std::cout<<x<<" "; std::cout<<"]"<<std::endl;

    // --- 7. Inverse and Determinant with cuSOLVER ---
    int work_size=0, *devInfo;
    CUDA_CHECK(cudaMalloc(&devInfo, sizeof(int)));
    float *d_inverse;
    CUDA_CHECK(cudaMalloc(&d_inverse, m*n*sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_inverse,h_A.data(),m*n*sizeof(float),cudaMemcpyHostToDevice));

    // Workspace size
    CUSOLVER_CHECK(cusolverDnSgetrf_bufferSize(cusolverH,m,n,d_inverse,m,&work_size));
    float *d_work; CUDA_CHECK(cudaMalloc(&d_work,work_size*sizeof(float)));

    int *d_ipiv; CUDA_CHECK(cudaMalloc(&d_ipiv,m*sizeof(int)));

    // --- LU factorization (A = L*U) ---
    CUSOLVER_CHECK(cusolverDnSgetrf(cusolverH,m,n,d_inverse,m,d_work,d_ipiv,devInfo));
    int h_info; CUDA_CHECK(cudaMemcpy(&h_info,devInfo,sizeof(int),cudaMemcpyDeviceToHost));

    // --- Determinant from U (product of diagonal elements) ---
    std::vector<float> h_LU(m*n);
    CUDA_CHECK(cudaMemcpy(h_LU.data(),d_inverse,m*n*sizeof(float),cudaMemcpyDeviceToHost));
    float det = h_LU[0]*h_LU[3] - h_LU[1]*h_LU[2]; // valid for 2x2
    std::cout<<"det(A) = "<<det<<std::endl;

    // --- Inverse via solving A * X = I ---
    float *d_I;
    CUDA_CHECK(cudaMalloc(&d_I, m*n*sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_I,h_I.data(),m*n*sizeof(float),cudaMemcpyHostToDevice));

    // Solve for inverse (columns of identity)
    CUSOLVER_CHECK(cusolverDnSgetrs(cusolverH,CUBLAS_OP_N,m,n,d_inverse,m,d_ipiv,d_I,m,devInfo));
    CUDA_CHECK(cudaMemcpy(h_C.data(),d_I,m*n*sizeof(float),cudaMemcpyDeviceToHost));

    std::cout<<"A^-1 = [ "; for(auto x:h_C) std::cout<<x<<" "; std::cout<<"]"<<std::endl;

    // Cleanup inverse workspace
    cudaFree(d_I); cudaFree(d_ipiv); cudaFree(d_work); cudaFree(devInfo); cudaFree(d_inverse);

    // Cleanup
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    cudaFree(d_inverse); cudaFree(d_ipiv); cudaFree(d_work); cudaFree(devInfo);
    cublasDestroy(cublasH); cusolverDnDestroy(cusolverH);

    return 0;
}

Writing 04_matrix_ops_lib.cu


In [17]:
!nvcc matrix_ops_lib.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o matrix_ops_lib

!./matrix_ops_lib

cc1plus: fatal error: matrix_ops_lib.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./matrix_ops_lib: No such file or directory


In [18]:
%%writefile 05_norms_distance_gpu.cu
#include <iostream>
#include <vector>
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <cmath>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<": "<<cudaGetErrorString(x)<<std::endl; exit(-1);}
#define CUBLAS_CHECK(x) if((x)!=CUBLAS_STATUS_SUCCESS){ \
    std::cerr<<"cuBLAS error at "<<__LINE__<<std::endl; exit(-1);}

// CUDA kernel to compute absolute values
__global__ void abs_kernel(const float* x, float* y, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if(idx < n) y[idx] = fabsf(x[idx]);
}

// CUDA kernel for reduction sum (simple version)
__global__ void reduce_sum(float* data, float* result, int n) {
    __shared__ float sdata[256];
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + tid;
    sdata[tid] = (idx < n) ? data[idx] : 0.0f;
    __syncthreads();

    // Reduce within block
    for(int s=blockDim.x/2; s>0; s>>=1) {
        if(tid < s) sdata[tid] += sdata[tid + s];
        __syncthreads();
    }

    // Write result of this block
    if(tid == 0) atomicAdd(result, sdata[0]);
}

int main() {
    int n = 3;
    std::vector<float> h_x = {1.0f, -2.0f, 3.0f};
    std::vector<float> h_y = {4.0f, 0.5f, -1.0f};

    float *d_x, *d_y;
    CUDA_CHECK(cudaMalloc(&d_x, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_y, n*sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_x, h_x.data(), n*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_y, h_y.data(), n*sizeof(float), cudaMemcpyHostToDevice));

    cublasHandle_t handle;
    CUBLAS_CHECK(cublasCreate(&handle));

    // --- L2 Norm (Euclidean) ---
    float l2_x, l2_y;
    CUBLAS_CHECK(cublasSnrm2(handle, n, d_x, 1, &l2_x));
    CUBLAS_CHECK(cublasSnrm2(handle, n, d_y, 1, &l2_y));
    std::cout << "L2 Norm of x = " << l2_x << "\n";
    std::cout << "L2 Norm of y = " << l2_y << "\n";

    // --- Cosine Similarity ---
    float dot;
    CUBLAS_CHECK(cublasSdot(handle, n, d_x, 1, d_y, 1, &dot));
    float cosine_sim = dot / (l2_x * l2_y);
    std::cout << "Cosine Similarity between x and y = " << cosine_sim << "\n";

    // --- L1 Norm (Manhattan) on GPU ---
    float *d_abs_x, *d_abs_y, *d_sum_x, *d_sum_y;
    float h_sum_x=0.0f, h_sum_y=0.0f;
    CUDA_CHECK(cudaMalloc(&d_abs_x, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_abs_y, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_sum_x, sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_sum_y, sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_sum_x, &h_sum_x, sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_sum_y, &h_sum_y, sizeof(float), cudaMemcpyHostToDevice));

    abs_kernel<<<1, n>>>(d_x, d_abs_x, n);
    abs_kernel<<<1, n>>>(d_y, d_abs_y, n);

    reduce_sum<<<1, 256>>>(d_abs_x, d_sum_x, n);
    reduce_sum<<<1, 256>>>(d_abs_y, d_sum_y, n);

    CUDA_CHECK(cudaMemcpy(&h_sum_x, d_sum_x, sizeof(float), cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(&h_sum_y, d_sum_y, sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "L1 Norm of x = " << h_sum_x << "\n";
    std::cout << "L1 Norm of y = " << h_sum_y << "\n";

    // Cleanup
    cudaFree(d_x); cudaFree(d_y);
    cudaFree(d_abs_x); cudaFree(d_abs_y);
    cudaFree(d_sum_x); cudaFree(d_sum_y);
    cublasDestroy(handle);

    return 0;
}

Writing 05_norms_distance_gpu.cu


In [19]:
!nvcc norms_distance_gpu.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o norms_distance_gpu

!./norms_distance_gpu

cc1plus: fatal error: norms_distance_gpu.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./norms_distance_gpu: No such file or directory


Systems of Linear Equations

In [20]:
%%writefile 06_solve_axb_cg.cu
#include <iostream>
#include <vector>
#include <cmath>
#include <cuda_runtime.h>
#include <cublas_v2.h>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<": "<<cudaGetErrorString(x)<<std::endl; return -1;}
#define CUBLAS_CHECK(x) if((x)!=CUBLAS_STATUS_SUCCESS){ \
    std::cerr<<"cuBLAS error at "<<__LINE__<<std::endl; return -1;}

int main() {
    // Example SPD matrix A (3x3) in COLUMN-MAJOR order:
    // A = [ 4 1 0
    //       1 3 1
    //       0 1 2 ]
    // Column-major storage (columns concatenated):
    std::vector<float> h_A = {
        4.0f, 1.0f, 0.0f,   // col 0
        1.0f, 3.0f, 1.0f,   // col 1
        0.0f, 1.0f, 2.0f    // col 2
    };
    // RHS b
    std::vector<float> h_b = {1.0f, 2.0f, 3.0f};

    int n = 3; // matrix size

    // Allocate device memory
    float *d_A = nullptr, *d_x = nullptr, *d_b = nullptr;
    float *d_r = nullptr, *d_p = nullptr, *d_Ap = nullptr;
    CUDA_CHECK(cudaMalloc(&d_A, n*n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_x, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_b, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_r, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_p, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_Ap, n*sizeof(float)));

    // Copy A and b to device
    CUDA_CHECK(cudaMemcpy(d_A, h_A.data(), n*n*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b, h_b.data(), n*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemset(d_x, 0, n*sizeof(float))); // initial x = 0

    // cuBLAS handle
    cublasHandle_t handle;
    CUBLAS_CHECK(cublasCreate(&handle));

    // Scalars for cuBLAS
    const float one = 1.0f;
    const float zero = 0.0f;
    //const float neg_one = -1.0f;

    // r0 = b - A*x0  (x0 = 0 -> r0 = b)
    CUBLAS_CHECK(cublasScopy(handle, n, d_b, 1, d_r, 1)); // r = b
    // p0 = r0
    CUBLAS_CHECK(cublasScopy(handle, n, d_r, 1, d_p, 1));

    // rsold = r' * r
    float rsold;
    CUBLAS_CHECK(cublasSdot(handle, n, d_r, 1, d_r, 1, &rsold));

    const int max_iter = 1000;
    const float tol = 1e-6f;
    int k;
    for (k = 0; k < max_iter; ++k) {
        // Ap = A * p  (use cublasSgemv: y = alpha*A*x + beta*y)
        // A is column-major, no transpose
        CUBLAS_CHECK(cublasSgemv(handle,
                                CUBLAS_OP_N,
                                n,      // rows
                                n,      // cols
                                &one,
                                d_A,    // A (n x n)
                                n,      // lda
                                d_p,    // x
                                1,
                                &zero,
                                d_Ap,
                                1));   // Ap = A * p

        // alpha = rsold / (p' * Ap)
        float pAp;
        CUBLAS_CHECK(cublasSdot(handle, n, d_p, 1, d_Ap, 1, &pAp));
        if (std::fabs(pAp) < 1e-12f) {
            std::cerr << "Break: p'Ap ~ 0 (numerical issue)\n";
            break;
        }
        float alpha = rsold / pAp;

        // x = x + alpha * p
        CUBLAS_CHECK(cublasSaxpy(handle, n, &alpha, d_p, 1, d_x, 1));

        // r = r - alpha * Ap  (use axpy with -alpha)
        float neg_alpha = -alpha;
        CUBLAS_CHECK(cublasSaxpy(handle, n, &neg_alpha, d_Ap, 1, d_r, 1));

        // rsnew = r' * r
        float rsnew;
        CUBLAS_CHECK(cublasSdot(handle, n, d_r, 1, d_r, 1, &rsnew));

        // Check convergence: sqrt(rsnew) = ||r||
        float rnorm = std::sqrt(rsnew);
        if (rnorm < tol) {
            std::cout << "Converged at iter " << k+1 << ", ||r|| = " << rnorm << "\n";
            break;
        }

        // p = r + (rsnew/rsold) * p
        float beta = rsnew / rsold;
        // p = beta * p
        CUBLAS_CHECK(cublasSscal(handle, n, &beta, d_p, 1));
        // p = p + r
        CUBLAS_CHECK(cublasSaxpy(handle, n, &one, d_r, 1, d_p, 1));

        rsold = rsnew;
    }

    // Copy solution x back to host
    std::vector<float> h_x(n);
    CUDA_CHECK(cudaMemcpy(h_x.data(), d_x, n*sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "Solution x (GPU CG): [ ";
    for (auto v : h_x) std::cout << v << " ";
    std::cout << "]\n";

    // Compute final residual norm on host for verification: r = b - A*x
    std::vector<float> Ax(n, 0.0f);
    // Compute Ax on host using h_A (column-major)
    for (int col = 0; col < n; ++col) {
        for (int row = 0; row < n; ++row) {
            Ax[row] += h_A[col*n + row] * h_x[col];
        }
    }
    std::vector<float> h_res(n);
    float res_norm = 0.0f;
    for (int i = 0; i < n; ++i) {
        h_res[i] = h_b[i] - Ax[i];
        res_norm += h_res[i] * h_res[i];
    }
    res_norm = std::sqrt(res_norm);
    std::cout << "Residual norm ||b - Ax|| = " << res_norm << std::endl;

    // Cleanup
    CUBLAS_CHECK(cublasDestroy(handle));
    cudaFree(d_A); cudaFree(d_x); cudaFree(d_b);
    cudaFree(d_r); cudaFree(d_p); cudaFree(d_Ap);

    return 0;
}

Writing 06_solve_axb_cg.cu


In [21]:
!nvcc solve_axb_cg.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o solve_axb_cg

!./solve_axb_cg

cc1plus: fatal error: solve_axb_cg.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./solve_axb_cg: No such file or directory


cuSolver

In [22]:
%%writefile 07_solve_axb_cusolver_lib.cu
#include <iostream>
#include <vector>
#include <cmath>
#include <cuda_runtime.h>
#include <cusolverDn.h>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<": "<<cudaGetErrorString(x)<<std::endl; return -1;}
#define CUSOLVER_CHECK(x) if((x)!=CUSOLVER_STATUS_SUCCESS){ \
    std::cerr<<"cuSOLVER error at "<<__LINE__<<std::endl; return -1;}

int main() {
    int n = 3; // matrix size

    // Example matrix A (row-major)
    std::vector<float> h_A = {4, 1, 0,
                              1, 3, 1,
                              0, 1, 2};
    // Right-hand side b
    std::vector<float> h_b = {1, 2, 3};

    // Device memory
    float *d_A = nullptr, *d_b = nullptr;
    int *d_ipiv = nullptr, *d_info = nullptr;
    CUDA_CHECK(cudaMalloc(&d_A, n*n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_b, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_ipiv, n*sizeof(int)));
    CUDA_CHECK(cudaMalloc(&d_info, sizeof(int)));

    // Copy A and b to device
    CUDA_CHECK(cudaMemcpy(d_A, h_A.data(), n*n*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b, h_b.data(), n*sizeof(float), cudaMemcpyHostToDevice));

    // cuSOLVER handle
    cusolverDnHandle_t cusolverH;
    CUSOLVER_CHECK(cusolverDnCreate(&cusolverH));

    // Workspace query
    int work_size = 0;
    CUSOLVER_CHECK(cusolverDnSgetrf_bufferSize(cusolverH, n, n, d_A, n, &work_size));

    float *d_work = nullptr;
    CUDA_CHECK(cudaMalloc(&d_work, work_size * sizeof(float)));

    // --- LU factorization ---
    CUSOLVER_CHECK(cusolverDnSgetrf(cusolverH, n, n, d_A, n, d_work, d_ipiv, d_info));

    int h_info = 0;
    CUDA_CHECK(cudaMemcpy(&h_info, d_info, sizeof(int), cudaMemcpyDeviceToHost));
    if (h_info != 0) {
        std::cerr << "LU factorization failed, info = " << h_info << std::endl;
        return -1;
    }

    // --- Solve Ax = b ---
    CUSOLVER_CHECK(cusolverDnSgetrs(cusolverH, CUBLAS_OP_N, n, 1, d_A, n, d_ipiv, d_b, n, d_info));
    CUDA_CHECK(cudaMemcpy(&h_info, d_info, sizeof(int), cudaMemcpyDeviceToHost));
    if (h_info != 0) {
        std::cerr << "Solve failed, info = " << h_info << std::endl;
        return -1;
    }

    // Copy solution back to host
    std::vector<float> h_x(n);
    CUDA_CHECK(cudaMemcpy(h_x.data(), d_b, n*sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "Solution x = [ ";
    for (auto v : h_x) std::cout << v << " ";
    std::cout << "]\n";

    // --- Compute residual norm ||b - Ax|| ---
    std::vector<float> res(n, 0.0f);
    for (int i = 0; i < n; i++) {
        float sum = 0.0f;
        for (int j = 0; j < n; j++) {
            sum += h_A[i*n + j] * h_x[j]; // row-major
        }
        res[i] = h_b[i] - sum;
    }
    float rnorm = 0.0f;
    for (int i = 0; i < n; i++) rnorm += res[i]*res[i];
    rnorm = std::sqrt(rnorm);
    std::cout << "Residual norm ||b - Ax|| = " << rnorm << std::endl;

    // Cleanup
    cudaFree(d_A); cudaFree(d_b); cudaFree(d_ipiv); cudaFree(d_info); cudaFree(d_work);
    cusolverDnDestroy(cusolverH);

    return 0;
}

Writing 07_solve_axb_cusolver_lib.cu


In [23]:
!nvcc solve_axb_cusolver_lib.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o solve_axb_cusolver_lib

!./solve_axb_cusolver_lib

cc1plus: fatal error: solve_axb_cusolver_lib.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./solve_axb_cusolver_lib: No such file or directory


In [24]:
%%writefile 08_olve_axb_cusolver_lib_01.cu
#include <iostream>
#include <vector>
#include <cmath>
#include <cuda_runtime.h>
#include <cusolverDn.h>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<": "<<cudaGetErrorString(x)<<std::endl; return -1;}
#define CUSOLVER_CHECK(x) if((x)!=CUSOLVER_STATUS_SUCCESS){ \
    std::cerr<<"cuSOLVER error at "<<__LINE__<<std::endl; return -1;}

int main() {
    int n = 3; // matrix size

    // Example matrix A (row-major)
    std::vector<float> h_A = {4, 1, 0,
                              1, 3, 1,
                              0, 1, 2};
    // Right-hand side b
    std::vector<float> h_b = {1, 2, 3};

    // Device memory
    float *d_A = nullptr, *d_b = nullptr;
    int *d_ipiv = nullptr, *d_info = nullptr;
    CUDA_CHECK(cudaMalloc(&d_A, n*n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_b, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_ipiv, n*sizeof(int)));
    CUDA_CHECK(cudaMalloc(&d_info, sizeof(int)));

    // Copy A and b to device
    CUDA_CHECK(cudaMemcpy(d_A, h_A.data(), n*n*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b, h_b.data(), n*sizeof(float), cudaMemcpyHostToDevice));

    // cuSOLVER handle
    cusolverDnHandle_t cusolverH;
    CUSOLVER_CHECK(cusolverDnCreate(&cusolverH));

    // Workspace query
    int work_size = 0;
    CUSOLVER_CHECK(cusolverDnSgetrf_bufferSize(cusolverH, n, n, d_A, n, &work_size));

    float *d_work = nullptr;
    CUDA_CHECK(cudaMalloc(&d_work, work_size * sizeof(float)));

    // --- LU factorization ---
    CUSOLVER_CHECK(cusolverDnSgetrf(cusolverH, n, n, d_A, n, d_work, d_ipiv, d_info));

    int h_info = 0;
    CUDA_CHECK(cudaMemcpy(&h_info, d_info, sizeof(int), cudaMemcpyDeviceToHost));
    if (h_info != 0) {
        std::cerr << "LU factorization failed, info = " << h_info << std::endl;
        return -1;
    }

    // --- Solve Ax = b ---
    CUSOLVER_CHECK(cusolverDnSgetrs(cusolverH, CUBLAS_OP_N, n, 1, d_A, n, d_ipiv, d_b, n, d_info));
    CUDA_CHECK(cudaMemcpy(&h_info, d_info, sizeof(int), cudaMemcpyDeviceToHost));
    if (h_info != 0) {
        std::cerr << "Solve failed, info = " << h_info << std::endl;
        return -1;
    }

    // Copy solution back to host
    std::vector<float> h_x(n);
    CUDA_CHECK(cudaMemcpy(h_x.data(), d_b, n*sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "Solution x = [ ";
    for (auto v : h_x) std::cout << v << " ";
    std::cout << "]\n";

    // --- Compute residual norm ||b - Ax|| in double precision ---
    std::vector<double> h_xd(n), h_Ad(n*n), h_bd(n);
    for (int i=0;i<n;i++) h_xd[i] = static_cast<double>(h_x[i]);
    for (int i=0;i<n*n;i++) h_Ad[i] = static_cast<double>(h_A[i]);
    for (int i=0;i<n;i++) h_bd[i] = static_cast<double>(h_b[i]);

    std::vector<double> res(n,0.0);
    for (int i=0;i<n;i++) {
        double sum=0.0;
        for (int j=0;j<n;j++) sum += h_Ad[i*n + j]*h_xd[j];
        res[i] = h_bd[i] - sum;
    }
    double rnorm=0.0;
    for (int i=0;i<n;i++) rnorm += res[i]*res[i];
    rnorm = std::sqrt(rnorm);

    std::cout << "Residual norm ||b - Ax|| = " << rnorm << std::endl;

    // Cleanup
    cudaFree(d_A); cudaFree(d_b); cudaFree(d_ipiv); cudaFree(d_info); cudaFree(d_work);
    cusolverDnDestroy(cusolverH);

    return 0;
}

Writing 08_olve_axb_cusolver_lib_01.cu


In [25]:
!nvcc solve_axb_cusolver_lib_01.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o solve_axb_cusolver_lib_01

!./solve_axb_cusolver_lib_01

cc1plus: fatal error: solve_axb_cusolver_lib_01.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./solve_axb_cusolver_lib_01: No such file or directory


Linear Transformations and Neural Network

In [26]:
%%writefile 09_linear_transform_nn_cuda.cu
#include <iostream>
#include <vector>
#include <cmath>
#include <cuda_runtime.h>
#include <cublas_v2.h>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<": "<<cudaGetErrorString(x)<<std::endl; return -1;}
#define CUBLAS_CHECK(x) if((x)!=CUBLAS_STATUS_SUCCESS){ \
    std::cerr<<"cuBLAS error at "<<__LINE__<<std::endl; return -1;}

// ReLU activation on host
void relu(std::vector<float> &v){
    for(auto &x:v) if(x<0) x=0;
}

int main() {
    // 3D points: 3 x 3 (each column = a point)
    std::vector<float> h_points = {1, 0, 1,
                                   0, 1, 1,
                                   0, 0, 1}; // 3 points
    int num_points = 3;

    // 3x3 rotation around Z by 90 deg + scaling
    float theta = 90.0f;
    float rad = theta * M_PI / 180.0f;
    float scale = 2.0f;
    std::vector<float> h_R = {
        scale * cos(rad), -scale * sin(rad), 0,
        scale * sin(rad),  scale * cos(rad), 0,
        0,                0,                scale
    };

    // Device memory
    float *d_points, *d_R, *d_result;
    CUDA_CHECK(cudaMalloc(&d_points, 3*num_points*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_R, 3*3*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_result, 3*num_points*sizeof(float)));

    CUDA_CHECK(cudaMemcpy(d_points, h_points.data(), 3*num_points*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_R, h_R.data(), 3*3*sizeof(float), cudaMemcpyHostToDevice));

    cublasHandle_t handle;
    CUBLAS_CHECK(cublasCreate(&handle));

    float alpha = 1.0f, beta = 0.0f;
    // Linear transformation: result = R * points
    CUBLAS_CHECK(cublasSgemm(handle,
                             CUBLAS_OP_N, CUBLAS_OP_N,
                             3, num_points, 3,
                             &alpha,
                             d_R, 3,
                             d_points, 3,
                             &beta,
                             d_result, 3));

    // Copy result back
    std::vector<float> h_result(3*num_points);
    CUDA_CHECK(cudaMemcpy(h_result.data(), d_result, 3*num_points*sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "Original 3D points:\n";
    for(int i=0;i<num_points;i++) std::cout << "(" << h_points[3*i] << "," << h_points[3*i+1] << "," << h_points[3*i+2] << ")\n";

    std::cout << "Transformed 3D points (rotation + scaling):\n";
    for(int i=0;i<num_points;i++) std::cout << "(" << h_result[3*i] << "," << h_result[3*i+1] << "," << h_result[3*i+2] << ")\n";

    // ---------------- Simple NN Forward ----------------
    // Input: 3 features x 3 samples
    // Layer1: 3x3 weight + bias
    std::vector<float> W1 = {0.5, -0.2, 0.1,
                             0.3, 0.8, -0.5,
                             -0.6, 0.1, 0.4};
    std::vector<float> b1 = {0.1, -0.1, 0.2};

    float *d_W1, *d_b1;
    CUDA_CHECK(cudaMalloc(&d_W1, 3*3*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_b1, 3*sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_W1, W1.data(), 3*3*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b1, b1.data(), 3*sizeof(float), cudaMemcpyHostToDevice));

    // Layer1: Y = W1*X + b
    CUBLAS_CHECK(cublasSgemm(handle,
                             CUBLAS_OP_N, CUBLAS_OP_N,
                             3, num_points, 3,
                             &alpha,
                             d_W1, 3,
                             d_result, 3,
                             &beta,
                             d_points, 3)); // reuse d_points for output

    // Copy result back for activation
    CUDA_CHECK(cudaMemcpy(h_result.data(), d_points, 3*num_points*sizeof(float), cudaMemcpyDeviceToHost));

    // Add bias and ReLU
    for(int i=0;i<num_points;i++)
        for(int j=0;j<3;j++)
            h_result[j + 3*i] += b1[j];
    relu(h_result);

    std::cout << "NN Layer1 output (after ReLU):\n";
    for(int i=0;i<num_points;i++) std::cout << "(" << h_result[3*i] << "," << h_result[3*i+1] << "," << h_result[3*i+2] << ")\n";

    // Cleanup
    cublasDestroy(handle);
    cudaFree(d_points); cudaFree(d_R); cudaFree(d_result);
    cudaFree(d_W1); cudaFree(d_b1);

    return 0;
}

Writing 09_linear_transform_nn_cuda.cu


In [27]:
!nvcc linear_transform_nn_cuda.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o linear_transform_nn_cuda

!./linear_transform_nn_cuda

cc1plus: fatal error: linear_transform_nn_cuda.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./linear_transform_nn_cuda: No such file or directory


Eigenvalues and Eigenvectors

In [28]:
%%writefile 10_eigen_cuda_lib.cu
#include <iostream>
#include <vector>
#include <cuda_runtime.h>
#include <cusolverDn.h>
#include <cmath>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<": "<<cudaGetErrorString(x)<<std::endl; return -1;}
#define CUSOLVER_CHECK(x) if((x)!=CUSOLVER_STATUS_SUCCESS){ \
    std::cerr<<"cuSOLVER error at "<<__LINE__<<std::endl; return -1;}

int main() {
    // Symmetric matrix 3x3
    std::vector<float> h_A = {4, 1, 1,
                              1, 3, 0,
                              1, 0, 2}; // row-major
    int n = 3;

    // Device memory
    float *d_A;
    CUDA_CHECK(cudaMalloc(&d_A, n*n*sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_A, h_A.data(), n*n*sizeof(float), cudaMemcpyHostToDevice));

    // cuSOLVER handle
    cusolverDnHandle_t solverH;
    CUSOLVER_CHECK(cusolverDnCreate(&solverH));

    // Workspace
    int lwork = 0;
    CUSOLVER_CHECK(cusolverDnSsyevd_bufferSize(solverH, CUSOLVER_EIG_MODE_VECTOR, CUBLAS_FILL_MODE_UPPER, n, d_A, n, nullptr, &lwork));

    float *d_W, *d_work;
    int *devInfo;
    CUDA_CHECK(cudaMalloc(&d_W, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_work, lwork*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&devInfo, sizeof(int)));

    // Compute eigenvalues and eigenvectors
    CUSOLVER_CHECK(cusolverDnSsyevd(solverH, CUSOLVER_EIG_MODE_VECTOR, CUBLAS_FILL_MODE_UPPER, n, d_A, n, d_W, d_work, lwork, devInfo));

    int h_info;
    CUDA_CHECK(cudaMemcpy(&h_info, devInfo, sizeof(int), cudaMemcpyDeviceToHost));
    if (h_info != 0) {
        std::cerr << "Eigen decomposition failed, info = " << h_info << std::endl;
        return -1;
    }

    // Copy results to host
    std::vector<float> h_W(n);
    std::vector<float> h_V(n*n);
    CUDA_CHECK(cudaMemcpy(h_W.data(), d_W, n*sizeof(float), cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(h_V.data(), d_A, n*n*sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "Eigenvalues:\n";
    for(auto w:h_W) std::cout << w << " ";
    std::cout << "\nEigenvectors (columns):\n";
    for(int i=0;i<n;i++){
        for(int j=0;j<n;j++) std::cout << h_V[j + i*n] << " ";
        std::cout << "\n";
    }

    // Cleanup
    cudaFree(d_A); cudaFree(d_W); cudaFree(d_work); cudaFree(devInfo);
    cusolverDnDestroy(solverH);

    return 0;
}

Writing 10_eigen_cuda_lib.cu


In [29]:
!nvcc eigen_cuda_lib.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o eigen_cuda_lib

!./eigen_cuda_lib

cc1plus: fatal error: eigen_cuda_lib.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./eigen_cuda_lib: No such file or directory


Decomposition

In [30]:
%%writefile 11_qr_cuda_lib.cu
#include <iostream>
#include <vector>
#include <cuda_runtime.h>
#include <cusolverDn.h>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<": "<<cudaGetErrorString(x)<<std::endl; return -1;}
#define CUSOLVER_CHECK(x) if((x)!=CUSOLVER_STATUS_SUCCESS){ \
    std::cerr<<"cuSOLVER error at "<<__LINE__<<std::endl; return -1;}

int main() {
    int m=3, n=2; // matrix size
    std::vector<float> h_A = {12, -51,
                               6, 167,
                              -4, 24}; // row-major

    float *d_A; CUDA_CHECK(cudaMalloc(&d_A, m*n*sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_A, h_A.data(), m*n*sizeof(float), cudaMemcpyHostToDevice));

    cusolverDnHandle_t solverH;
    CUSOLVER_CHECK(cusolverDnCreate(&solverH));

    int lwork=0;
    CUSOLVER_CHECK(cusolverDnSgeqrf_bufferSize(solverH, m, n, d_A, m, &lwork));

    float *d_tau, *d_work; int *devInfo;
    CUDA_CHECK(cudaMalloc(&d_tau, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_work, lwork*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&devInfo, sizeof(int)));

    // Compute QR factorization
    CUSOLVER_CHECK(cusolverDnSgeqrf(solverH, m, n, d_A, m, d_tau, d_work, lwork, devInfo));

    // Copy back results
    std::vector<float> h_R(m*n);
    std::vector<float> h_tau(n);
    CUDA_CHECK(cudaMemcpy(h_R.data(), d_A, m*n*sizeof(float), cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(h_tau.data(), d_tau, n*sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "R (upper-triangular part of A):\n";
    for(int i=0;i<m;i++){
        for(int j=0;j<n;j++) std::cout<<h_R[i + j*m]<<" ";
        std::cout<<std::endl;
    }

    cudaFree(d_A); cudaFree(d_tau); cudaFree(d_work); cudaFree(devInfo);
    cusolverDnDestroy(solverH);
    return 0;
}

Writing 11_qr_cuda_lib.cu


In [31]:
!nvcc qr_cuda_lib.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o qr_cuda_lib

!./qr_cuda_lib

cc1plus: fatal error: qr_cuda_lib.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./qr_cuda_lib: No such file or directory


In [32]:
%%writefile 12_svd_cuda_lib.cu
#include <iostream>
#include <vector>
#include <cuda_runtime.h>
#include <cusolverDn.h>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<": "<<cudaGetErrorString(x)<<std::endl; return -1;}
#define CUSOLVER_CHECK(x) if((x)!=CUSOLVER_STATUS_SUCCESS){ \
    std::cerr<<"cuSOLVER error at "<<__LINE__<<std::endl; return -1;}

int main() {
    int m=3, n=2;
    std::vector<float> h_A = {1, 0,
                              0, 1,
                              1, 1}; // row-major m x n

    float *d_A;
    CUDA_CHECK(cudaMalloc(&d_A, m*n*sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_A, h_A.data(), m*n*sizeof(float), cudaMemcpyHostToDevice));

    cusolverDnHandle_t solverH;
    CUSOLVER_CHECK(cusolverDnCreate(&solverH));

    int lwork=0;
    CUSOLVER_CHECK(cusolverDnSgesvd_bufferSize(solverH, m, n, &lwork));

    float *d_S, *d_U, *d_VT, *d_work; int *devInfo;
    CUDA_CHECK(cudaMalloc(&d_S, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_U, m*m*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_VT, n*n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_work, lwork*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&devInfo, sizeof(int)));

    signed char jobu = 'A', jobvt = 'A';
    CUSOLVER_CHECK(cusolverDnSgesvd(solverH, jobu, jobvt, m, n, d_A, m, d_S, d_U, m, d_VT, n, d_work, lwork, nullptr, devInfo));

    std::vector<float> h_S(n);
    CUDA_CHECK(cudaMemcpy(h_S.data(), d_S, n*sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "Singular values:\n";
    for(auto s:h_S) std::cout << s << " ";
    std::cout << std::endl;

    cudaFree(d_A); cudaFree(d_S); cudaFree(d_U); cudaFree(d_VT); cudaFree(d_work); cudaFree(devInfo);
    cusolverDnDestroy(solverH);

    return 0;
}

Writing 12_svd_cuda_lib.cu


In [33]:
!nvcc svd_cuda_lib.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o svd_cuda_lib

!./svd_cuda_lib

cc1plus: fatal error: svd_cuda_lib.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./svd_cuda_lib: No such file or directory


Orthogonality

In [34]:
%%writefile 13_orth_proj_cuda.cu
#include <iostream>
#include <vector>
#include <cmath>
#include <cuda_runtime.h>
#include <cublas_v2.h>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<": "<<cudaGetErrorString(x)<<std::endl; return -1;}
#define CUBLAS_CHECK(x) if((x)!=CUBLAS_STATUS_SUCCESS){ \
    std::cerr<<"cuBLAS error at "<<__LINE__<<std::endl; return -1;}

// Gram-Schmidt orthonormalization on host
void gram_schmidt(std::vector<float>& v1, std::vector<float>& v2) {
    // Normalize v1
    float norm1 = std::sqrt(v1[0]*v1[0]+v1[1]*v1[1]+v1[2]*v1[2]);
    for(int i=0;i<3;i++) v1[i] /= norm1;

    // v2 = v2 - proj_v1(v2)
    float dot = v1[0]*v2[0] + v1[1]*v2[1] + v1[2]*v2[2];
    for(int i=0;i<3;i++) v2[i] -= dot*v1[i];

    // Normalize v2
    float norm2 = std::sqrt(v2[0]*v2[0]+v2[1]*v2[1]+v2[2]*v2[2]);
    for(int i=0;i<3;i++) v2[i] /= norm2;
}

int main() {
    // 3D vectors spanning subspace
    std::vector<float> v1 = {1,1,0};
    std::vector<float> v2 = {1,0,1};
    gram_schmidt(v1,v2);

    // Vector to project
    std::vector<float> b = {3,2,1};

    // Device memory
    float *d_Q, *d_b, *d_proj;
    CUDA_CHECK(cudaMalloc(&d_Q, 3*2*sizeof(float))); // 2 orthonormal vectors
    CUDA_CHECK(cudaMalloc(&d_b, 3*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_proj, 3*sizeof(float)));

    // Copy Q and b
    std::vector<float> h_Q = {v1[0], v1[1], v1[2],
                              v2[0], v2[1], v2[2]}; // column-major
    CUDA_CHECK(cudaMemcpy(d_Q, h_Q.data(), 3*2*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b, b.data(), 3*sizeof(float), cudaMemcpyHostToDevice));

    cublasHandle_t handle;
    CUBLAS_CHECK(cublasCreate(&handle));

    float alpha=1.0f, beta=0.0f;
    // proj = Q * (Q^T * b)
    float *d_temp;
    CUDA_CHECK(cudaMalloc(&d_temp, 2*sizeof(float))); // 2x1 vector
    // d_temp = Q^T * b
    CUBLAS_CHECK(cublasSgemv(handle, CUBLAS_OP_T, 3, 2, &alpha, d_Q, 3, d_b, 1, &beta, d_temp, 1));
    // d_proj = Q * d_temp
    CUBLAS_CHECK(cublasSgemv(handle, CUBLAS_OP_N, 3, 2, &alpha, d_Q, 3, d_temp, 1, &beta, d_proj, 1));

    // Copy projection back
    std::vector<float> h_proj(3);
    CUDA_CHECK(cudaMemcpy(h_proj.data(), d_proj, 3*sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "Projection of b onto subspace spanned by v1 and v2:\n";
    std::cout << "(" << h_proj[0] << ", " << h_proj[1] << ", " << h_proj[2] << ")\n";

    cudaFree(d_Q); cudaFree(d_b); cudaFree(d_proj); cudaFree(d_temp);
    cublasDestroy(handle);

    return 0;
}

Writing 13_orth_proj_cuda.cu


In [35]:
!nvcc orth_proj_cuda.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o orth_proj_cuda

!./orth_proj_cuda

cc1plus: fatal error: orth_proj_cuda.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./orth_proj_cuda: No such file or directory


In [36]:
%%writefile 14_orth_proj_lib.cu
#include <iostream>
#include <vector>
#include <cuda_runtime.h>
#include <cusolverDn.h>
#include <cublas_v2.h>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<": "<<cudaGetErrorString(x)<<std::endl; return -1;}
#define CUSOLVER_CHECK(x) if((x)!=CUSOLVER_STATUS_SUCCESS){ \
    std::cerr<<"cuSOLVER error at "<<__LINE__<<std::endl; return -1;}
#define CUBLAS_CHECK(x) if((x)!=CUBLAS_STATUS_SUCCESS){ \
    std::cerr<<"cuBLAS error at "<<__LINE__<<std::endl; return -1;}

int main() {
    int m = 3; // dimension
    int n = 2; // number of vectors spanning subspace

    // Columns are vectors spanning subspace
    std::vector<float> h_A = {1,1,0,  // v1
                              1,0,1}; // v2
    // Row-major to column-major
    std::vector<float> h_A_col(m*n);
    for(int j=0;j<n;j++)
        for(int i=0;i<m;i++)
            h_A_col[i + j*m] = h_A[j*m + i];

    // Vector to project
    std::vector<float> h_b = {3,2,1};

    float *d_A, *d_tau, *d_b, *d_proj;
    CUDA_CHECK(cudaMalloc(&d_A, m*n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_tau, n*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_b, m*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_proj, m*sizeof(float)));

    CUDA_CHECK(cudaMemcpy(d_A, h_A_col.data(), m*n*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b, h_b.data(), m*sizeof(float), cudaMemcpyHostToDevice));

    cusolverDnHandle_t solverH;
    CUSOLVER_CHECK(cusolverDnCreate(&solverH));

    int lwork = 0;
    CUSOLVER_CHECK(cusolverDnSgeqrf_bufferSize(solverH, m, n, d_A, m, &lwork));

    float *d_work;
    int *devInfo;
    CUDA_CHECK(cudaMalloc(&d_work, lwork*sizeof(float)));
    CUDA_CHECK(cudaMalloc(&devInfo, sizeof(int)));

    // Compute QR decomposition: d_A -> Q (orthonormal vectors in first n columns)
    CUSOLVER_CHECK(cusolverDnSgeqrf(solverH, m, n, d_A, m, d_tau, d_work, lwork, devInfo));

    // Generate explicit Q
    CUSOLVER_CHECK(cusolverDnSorgqr(solverH, m, n, n, d_A, m, d_tau, d_work, lwork, devInfo));

    // Now d_A contains Q (m x n), orthonormal columns
    cublasHandle_t handle;
    CUBLAS_CHECK(cublasCreate(&handle));

    float alpha=1.0f, beta=0.0f;
    float *d_temp;
    CUDA_CHECK(cudaMalloc(&d_temp, n*sizeof(float)));

    // temp = Q^T * b
    CUBLAS_CHECK(cublasSgemv(handle, CUBLAS_OP_T, m, n, &alpha, d_A, m, d_b, 1, &beta, d_temp, 1));
    // proj = Q * temp
    CUBLAS_CHECK(cublasSgemv(handle, CUBLAS_OP_N, m, n, &alpha, d_A, m, d_temp, 1, &beta, d_proj, 1));

    // Copy projection back
    std::vector<float> h_proj(m);
    CUDA_CHECK(cudaMemcpy(h_proj.data(), d_proj, m*sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "Projection of b onto subspace spanned by Q:\n";
    for(int i=0;i<m;i++) std::cout << h_proj[i] << " ";
    std::cout << std::endl;

    // Cleanup
    cudaFree(d_A); cudaFree(d_tau); cudaFree(d_b); cudaFree(d_proj); cudaFree(d_work); cudaFree(d_temp); cudaFree(devInfo);
    cusolverDnDestroy(solverH);
    cublasDestroy(handle);

    return 0;
}

Writing 14_orth_proj_lib.cu


In [37]:
!nvcc orth_proj_lib.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o orth_proj_lib

!./orth_proj_lib

cc1plus: fatal error: orth_proj_lib.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./orth_proj_lib: No such file or directory


In [38]:
%%writefile 15_rank_nullspace_cublas.cu
#include <iostream>
#include <vector>
#include <cuda_runtime.h>
#include <cublas_v2.h>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<": "<<cudaGetErrorString(x)<<std::endl; return -1;}
#define CUBLAS_CHECK(x) if((x)!=CUBLAS_STATUS_SUCCESS){ \
    std::cerr<<"cuBLAS error at "<<__LINE__<<std::endl; return -1;}

int main() {
    // Matrix dimensions
    int m = 2, n = 3;

    // Column-major matrix A
    std::vector<float> h_A = {
        1,4,   // col 0
        2,5,   // col 1
        3,6    // col 2
    };

    // Device memory
    float *d_A, *d_AtA;
    CUDA_CHECK(cudaMalloc(&d_A, m*n*sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_A, h_A.data(), m*n*sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMalloc(&d_AtA, n*n*sizeof(float)));

    // cuBLAS handle
    cublasHandle_t handle;
    CUBLAS_CHECK(cublasCreate(&handle));

    float alpha = 1.0f, beta = 0.0f;

    // Compute AtA = A^T * A
    // A: m x n, AtA: n x n
    CUBLAS_CHECK(cublasSgemm(handle, CUBLAS_OP_T, CUBLAS_OP_N,
                             n, n, m, &alpha,
                             d_A, m,
                             d_A, m,
                             &beta, d_AtA, n));

    // Copy AtA to host
    std::vector<float> h_AtA(n*n);
    CUDA_CHECK(cudaMemcpy(h_AtA.data(), d_AtA, n*n*sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "A^T * A = \n";
    for(int i=0;i<n;i++){
        for(int j=0;j<n;j++) std::cout << h_AtA[i + j*n] << " "; // column-major
        std::cout << "\n";
    }

    // Approximate rank by counting non-zero diagonals
    float eps = 1e-6;
    int rank = 0;
    for(int i=0;i<n;i++) if(fabs(h_AtA[i + i*n]) > eps) rank++;
    std::cout << "Approximate Rank of A = " << rank << std::endl;

    // Cleanup
    cudaFree(d_A); cudaFree(d_AtA);
    cublasDestroy(handle);

    return 0;
}

Writing 15_rank_nullspace_cublas.cu


In [39]:
!nvcc rank_nullspace_cublas.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o rank_nullspace_cublas

!./rank_nullspace_cublas

cc1plus: fatal error: rank_nullspace_cublas.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./rank_nullspace_cublas: No such file or directory


In [40]:
%%writefile 16_nullspace_cublas.cu
#include <iostream>
#include <vector>
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <cmath>

#define CUDA_CHECK(x) if((x)!=cudaSuccess){ \
    std::cerr<<"CUDA error at "<<__LINE__<<": "<<cudaGetErrorString(x)<<std::endl; exit(-1);}
#define CUBLAS_CHECK(x) if((x)!=CUBLAS_STATUS_SUCCESS){ \
    std::cerr<<"cuBLAS error at "<<__LINE__<<std::endl; exit(-1);}

// Compute Euclidean norm of a vector on GPU
float gpu_norm(cublasHandle_t handle, float* d_x, int len) {
    float result;
    CUBLAS_CHECK(cublasSnrm2(handle, len, d_x, 1, &result));
    return result;
}

// Subtract projection: x = x - (v^T x) * v
void gpu_subtract_projection(cublasHandle_t handle, float* d_x, float* d_v, int len) {
    float dot;
    CUBLAS_CHECK(cublasSdot(handle, len, d_v, 1, d_x, 1, &dot));
    float alpha = -dot;
    CUBLAS_CHECK(cublasSaxpy(handle, len, &alpha, d_v, 1, d_x, 1));
}

int main() {
    int m = 2, n = 3;
    std::vector<float> h_A = {
        1,4,  // col 0
        2,5,  // col 1
        3,6   // col 2
    };

    float *d_A;
    CUDA_CHECK(cudaMalloc(&d_A, m*n*sizeof(float)));
    CUDA_CHECK(cudaMemcpy(d_A, h_A.data(), m*n*sizeof(float), cudaMemcpyHostToDevice));

    cublasHandle_t handle;
    CUBLAS_CHECK(cublasCreate(&handle));

    // Null space vector
    float *d_x;
    CUDA_CHECK(cudaMalloc(&d_x, n*sizeof(float)));

    // Initial guess (non-zero vector)
    std::vector<float> h_x = {1.0f, -2.0f, 1.0f};
    CUDA_CHECK(cudaMemcpy(d_x, h_x.data(), n*sizeof(float), cudaMemcpyHostToDevice));

    // Orthogonalize w.r.t columns of A (Gram-Schmidt)
    for(int j=0; j<m; j++) {
        float *d_col = d_A + j; // column-major stride
        gpu_subtract_projection(handle, d_x, d_col, n);
    }

    // Normalize
    float norm = gpu_norm(handle, d_x, n);
    float alpha = 1.0f/norm;
    CUBLAS_CHECK(cublasSscal(handle, n, &alpha, d_x, 1));

    // Copy back to host
    CUDA_CHECK(cudaMemcpy(h_x.data(), d_x, n*sizeof(float), cudaMemcpyDeviceToHost));

    std::cout << "Null space basis vector (Ax=0): [ ";
    for(auto v : h_x) std::cout << v << " ";
    std::cout << "]\n";

    // Cleanup
    cudaFree(d_A); cudaFree(d_x);
    cublasDestroy(handle);

    return 0;
}

Writing 16_nullspace_cublas.cu


In [41]:
!nvcc nullspace_cublas.cu -I/usr/include -L/usr/lib/x86_64-linux-gnu -lcutensor -lcusolver -lcublasLt -lcublas -o nullspace_cublas

!./nullspace_cublas

cc1plus: fatal error: nullspace_cublas.cu: No such file or directory
compilation terminated.
/bin/bash: line 1: ./nullspace_cublas: No such file or directory


In [44]:
!mkdir cu_c++_code

In [45]:
!cp *.cu cu_c++_code/

In [46]:
!zip -r cu_c++_code.zip cu_c++_code/

  adding: cu_c++_code/ (stored 0%)
  adding: cu_c++_code/04_matrix_ops_lib.cu (deflated 76%)
  adding: cu_c++_code/02_vector_ops.cu (deflated 72%)
  adding: cu_c++_code/01_cutensor_basics.cu (deflated 63%)
  adding: cu_c++_code/03_vector_ops_lib.cu (deflated 72%)
  adding: cu_c++_code/00_test_cutensor.cu (deflated 43%)
  adding: cu_c++_code/12_svd_cuda_lib.cu (deflated 62%)
  adding: cu_c++_code/09_linear_transform_nn_cuda.cu (deflated 72%)
  adding: cu_c++_code/15_rank_nullspace_cublas.cu (deflated 59%)
  adding: cu_c++_code/07_solve_axb_cusolver_lib.cu (deflated 67%)
  adding: cu_c++_code/05_norms_distance_gpu.cu (deflated 67%)
  adding: cu_c++_code/13_orth_proj_cuda.cu (deflated 62%)
  adding: cu_c++_code/11_qr_cuda_lib.cu (deflated 61%)
  adding: cu_c++_code/08_olve_axb_cusolver_lib_01.cu (deflated 67%)
  adding: cu_c++_code/10_eigen_cuda_lib.cu (deflated 63%)
  adding: cu_c++_code/06_solve_axb_cg.cu (deflated 68%)
  adding: cu_c++_code/16_nullspace_cublas.cu (deflated 60%)
  addin

In [47]:
from google.colab import files
files.download('cu_c++_code.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>